In [1]:
# import libraries
import pandas as pd

In [2]:
# read covid csv data
covid = pd.read_csv('last7days-by-modzcta.csv')

In [3]:
covid

,modzcta,modzcta_name,label,lat,lon,percentpositivity_7day,people_tested,people_positive,median_daily_test_rate,adequately_tested,daterange
0,10001,Chelsea/NoMad/West Chelsea,"10001, 10118",40.750693,-73.997137,0.46,658,3,369.4,Yes,June 19-June 25
1,10002,Chinatown/Lower East Side,10002,40.715781,-73.986176,0.25,1212,3,204.5,No,June 19-June 25
2,10003,East Village/Gramercy/Greenwich Village,10003,40.731825,-73.989164,0.29,1051,3,331.6,Yes,June 19-June 25
3,10004,Financial District,10004,40.703675,-74.013106,0.00,79,0,471.0,Yes,June 19-June 25
4,10005,Financial District,10005,40.706092,-74.008861,0.37,271,1,536.7,Yes,June 19-June 25
...,...,...,...,...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,11691,40.601330,-73.761357,0.49,1228,6,201.9,No,June 19-June 25
173,11692,Arverne/Edgemere,11692,40.592794,-73.792203,0.53,377,2,242.5,No,June 19-June 25
174,11693,Arverne/Broad Channel,11693,40.603699,-73.821926,0.41,242,1,205.2,No,June 19-June 25
175,11694,Belle Harbor-Neponsit/Rockaway Park,11694,40.577599,-73.844553,0.47,423,2,192.5,No,June 19-June 25


In [4]:
# filter out necessary covid data
nyc_cov = covid[['modzcta_name', 'median_daily_test_rate']]

In [7]:
# rename covid data columns for consistency
nyc_cov.rename(columns={'modzcta_name':'name'}, inplace=True)

C:\Users\eric_\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
nyc_cov

,name,median_daily_test_rate
0,Chelsea/NoMad/West Chelsea,369.4
1,Chinatown/Lower East Side,204.5
2,East Village/Gramercy/Greenwich Village,331.6
3,Financial District,471.0
4,Financial District,536.7
...,...,...
172,Edgemere/Far Rockaway,201.9
173,Arverne/Edgemere,242.5
174,Arverne/Broad Channel,205.2
175,Belle Harbor-Neponsit/Rockaway Park,192.5


In [9]:
# double explode method to get rid of / and - in names to split aggregated neighborhoods
nyc_cov['name'] = nyc_cov['name'].str.split('/')

<ipython-input-9-b4dca63aefea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_cov['name'] = nyc_cov['name'].str.split('/')


In [11]:
nyc_cov1 = nyc_cov.explode('name')

In [13]:
nyc_cov1['name'] = nyc_cov1['name'].str.split('-')

In [15]:
nyc_cov2 = nyc_cov1.explode('name')

In [19]:
nyc_cov2.sort_values(by='name', inplace=True)

In [27]:
nyc_cov3 = nyc_cov2.groupby(by='name', as_index=False).sum()

In [22]:
# function to fully print entire dataframe for visual
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [137]:
print_full(nyc_cov3)

                          name  median_daily_test_rate
1                     Allerton                   511.5
3                     Annadale                   263.7
4                     Arrochar                   317.3
5                      Arverne                   447.7
6                      Astoria                   666.2
7                      Astoria                   989.1
8                   Auburndale                   380.3
9                   Bath Beach                   388.2
10           Battery Park City                   830.8
11                   Bay Ridge                   285.9
12                  Baychester                   261.6
13                     Bayside                   428.8
14                     Bayside                   330.5
15                Bedford Park                  1179.1
16                Belle Harbor                   192.5
17                   Bellerose                   605.8
18                     Belmont                   487.2
19        

In [122]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
nyc_cov3.at[223, 'name'] = 'Williamsburg'

In [119]:
# remove columns that do not exist in original nyc_geo data
indexname = nyc_cov3[nyc_cov3['name'] == 'West Harlem'].index
nyc_cov3.drop(indexname, inplace=True)

In [124]:
# grouping duplicate neighborhoods and summing the median daily test rates
# this is because some of the neighborhoods in the covid data were separate but were considered as one neighborhood in our original nyc_geo data
nyc_cov4 = nyc_cov3.groupby(by='name', as_index=False).sum()

In [125]:
nyc_cov4

,name,median_daily_test_rate
0,Allerton,511.5
1,Annadale,263.7
2,Arrochar,317.3
3,Arverne,447.7
4,Astoria,1655.3
...,...,...
200,Woodhaven,222.2
201,Woodlawn,260.1
202,Woodrow,297.5
203,Woodside,243.9


In [126]:
# read nyc_upp data
nyc_upp = pd.read_csv('nyc_upp_data.csv')

In [128]:
# merge covid data to nyc_upp data
nyc_uppc = nyc_upp.merge(nyc_cov4, how='outer', on='name')

In [130]:
# about one-third of our data for covid is null
# this may be due to neighborhood naming differences between data sets
# could also be due to lack of recording data in some neighborhoods
nyc_uppc.isnull().sum()

name                       0
longitude                  0
latitude                   0
uber_count                 0
avg_price                  0
adj_pop                    0
median_daily_test_rate    97
dtype: int64

In [139]:
nyc_cov4['median_daily_test_rate'].mean()

482.5575609756098

In [140]:
# choosing to fill na covid values with the mean of covid values
nyc_uppc.fillna(value=482.5, inplace=True)

In [142]:
nyc_uppc.isnull().sum()

name                      0
longitude                 0
latitude                  0
uber_count                0
avg_price                 0
adj_pop                   0
median_daily_test_rate    0
dtype: int64

In [143]:
# read ela scores data
ela = pd.read_csv('ELA_Scores.csv')

In [145]:
# filter out necessary ela columns
ela = ela[['name', 'avg_test_score']]

In [172]:
# dropping duplicates in ela data 
ela.drop_duplicates(subset='name', inplace=True, keep='first')

<ipython-input-172-7d2c402084db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ela.drop_duplicates(subset='name', inplace=True, keep='first')


In [176]:
# merge ela data to create completed dataset
nyc_data = nyc_uppc.merge(ela, how='left', on='name')

In [178]:
# save completed dataset as nyc_data.csv
nyc_data.to_csv('nyc_data.csv', index=False)